# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [29]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [30]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [31]:
# Your code here
scaler = StandardScaler()

# Scale X_train and X_val using StandardScaler
scaler.fit(X_train)


# Ensure X_train and X_val are scaled DataFrames
# (hint: you can set the columns using X.columns)
X_train_scaled  = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)

X_train_scaled = pd.DataFrame(X_train_scaled, columns = X.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns = X.columns)



In [32]:
X_train_scaled.head()

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces
0,-0.114710,-0.099842,-0.509252,-0.639316,-0.804789,1.261552,0.499114,0.250689,0.327629,-0.994820
1,-0.176719,0.632038,-0.509252,0.838208,0.641608,-0.808132,-0.247249,-0.365525,0.079146,-0.994820
2,-0.246336,-0.831723,1.304613,-0.012560,-0.329000,-0.808132,-0.944766,-0.981739,-1.105931,-0.994820
3,-0.378617,-0.831723,1.304613,-0.339045,-0.609036,-0.808132,-1.146010,-0.981739,-1.134602,0.588023
4,-0.010898,-1.563603,1.304613,-2.531499,-1.315922,0.550523,-0.481708,0.250689,-2.281450,-0.994820


In [33]:
X_val_scaled.head()

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces
0,-0.491264,-0.099842,0.397681,-0.248487,-0.598161,-0.808132,-1.138195,-0.981739,-0.178895,-0.994820
1,-0.619291,0.632038,-0.509252,-0.205591,-0.549222,0.849426,0.327177,0.250689,-0.465607,-0.994820
2,0.240165,-0.099842,0.397681,-0.639316,-1.044043,0.722619,-0.137834,-0.365525,-0.427379,-0.994820
3,0.037695,2.095798,-3.230050,1.891539,1.843314,-0.808132,0.616344,0.866903,1.703847,0.588023
4,-0.147924,0.632038,-0.509252,-1.616388,-0.995104,0.709032,-0.114388,0.250689,-0.408265,0.588023


In [34]:
# Your code here
lr = LinearRegression()

# Create a LinearRegression model and fit it on scaled training data
model = lr.fit(X_train_scaled, y_train)
# Calculate a baseline r-squared score on training data
model.score(X_train,y_train)


-3413419.543320529

## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [35]:
# Your code here

from itertools import combinations

# Set up data structure
interaction_results = []

# Find combinations of columns and loop over them
for col1,col2 in combinations(X.columns,2):
    
    # Make copies of X_train and X_val
    
    X_train_interaction = X_train_scaled.copy()
    y = y_train
    X_val_interaction = X_val_scaled.copy()
    y_val = y_val.copy()
    lr = LinearRegression()
    lr.fit(X_train_scaled, y_train)
    baseline_r2 = lr.score(X_train_scaled, y_train)
    
    # Add interaction term to data

    X_train_interaction[f'{col1}_x_{col2}'] = X_train_interaction[col1]* X_train_interaction[col2]
    
    X_val_interaction[f'{col1}_x_{col2}'] = X_val_interaction[col1]* X_val_interaction[col2]

    # Find r-squared score (fit on training data, evaluate on validation data)

    model2 =lr.fit(X_train_interaction,y)

    r2_with_interaction = lr.score(X_val_interaction,y_val)

    # Append to data structure
    interaction_results.append({
        'interaction': f'{col1} * {col2}',
        'r2_score': r2_with_interaction,
        'improvement': r2_with_interaction - baseline_r2
    })
    
# Sort and subset the data structure to find the top 7
interaction_df = pd.DataFrame(interaction_results)
top_interactions = interaction_df.nlargest(7,'improvement')
top_interactions

,interaction,r2_score,improvement
3,LotArea * 1stFlrSF,0.721111,-0.065724
2,LotArea * TotalBsmtSF,0.707165,-0.079670
5,LotArea * GrLivArea,0.669098,-0.117736
8,LotArea * Fireplaces,0.652970,-0.133865
36,2ndFlrSF * TotRmsAbvGrd,0.647299,-0.139535
17,OverallCond * TotalBsmtSF,0.642902,-0.143932
12,OverallQual * 2ndFlrSF,0.642232,-0.144602


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [36]:
# Your code here
X_train.columns = X_train.columns.str.strip()
X_val.columns = X_val.columns.str.strip()

# Loop over top 7 interactions
for index,row in top_interactions.iterrows():
    # Extract column names from data structure
    interaction_terms = row['interaction'].split('*')
    col1 = interaction_terms[0].strip()
    col2 = interaction_terms[1].strip()
    # Construct new column name
    new_column_name = f'{col1}_{col2}'
    # Add new column to X_train and X_val
    X_train[new_column_name] = X_train[col1] * X_train[col2]
    X_val[new_column_name] = X_val[col1] * X_val[col2]


In [37]:
X_train.head()

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,LotArea_GrLivArea,LotArea_Fireplaces,2ndFlrSF_TotRmsAbvGrd,OverallCond_TotalBsmtSF,OverallQual_2ndFlrSF
518,9531,6,5,794,882,914,1796,7,546,0,8406342,7567614,17117676,0,6398,3970,5484
236,8773,7,5,1414,1414,0,1414,6,494,0,12405022,12405022,12405022,0,0,7070,0
38,7922,5,7,1057,1057,0,1057,5,246,0,8373554,8373554,8373554,0,0,7399,0
1034,6305,5,7,920,954,0,954,5,240,1,6014970,5800600,6014970,6305,0,6440,0
520,10800,4,7,0,694,600,1294,7,0,0,7495200,0,13975200,0,4200,0,2400


In [38]:
X_val.head()

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,LotArea_GrLivArea,LotArea_Fireplaces,2ndFlrSF_TotRmsAbvGrd,OverallCond_TotalBsmtSF,OverallQual_2ndFlrSF
1431,4928,6,6,958,958,0,958,5,440,0,4721024,4721024,4721024,0,0,5748,0
1304,3363,7,5,976,976,732,1708,7,380,0,3282288,3282288,5744004,0,5124,4880,5124
50,13869,6,6,794,794,676,1470,6,388,0,11011986,11011986,20387430,0,4056,4764,4056
378,11394,9,2,1856,1856,0,1856,8,834,1,21147264,21147264,21147264,11394,0,3712,0
1124,9125,7,5,384,812,670,1482,7,392,1,7409500,3504000,13523250,9125,4690,1920,4690


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [39]:
 
# Your code here
# Set up data structure
poly_results= []
degrees = [2,3,4]

# Loop over all columns

for col in X_train_scaled.columns:
    
    # Loop over degrees 2, 3, 4
    
     for degree in degrees:  
         
        # Make a copy of X_train and X_val
         
        X_train_poly = X_train_scaled.copy()
        y = y_train
        X_val_poly = X_val_scaled.copy()
        y_val = y_val.copy()
         
        # Instantiate PolynomialFeatures with relevant degree
         
        poly = PolynomialFeatures(degree = degree)
         
        # Fit polynomial to column and transform column
        
        X_train_col = X_train_poly[[col]]
        X_val_col = X_val_poly[[col]]
    
        # Hint: use the notation df[[column_name]] to get the right shape
    
        # Hint: convert the result to a DataFrame
        col_X_train_poly = poly.fit_transform(X_train_col)
        col_X_val_poly = poly.transform( X_val_col)
         
        # Add polynomial to data
        poly_feature_names = poly.get_feature_names_out([col])
        X_train_poly_df = pd.DataFrame(col_X_train_poly,columns = poly_feature_names)
        X_val_poly_df = pd.DataFrame(col_X_val_poly,columns = poly_feature_names)
         
        # Hint: use pd.concat since you're combining two DataFrames
         # Hint: drop the column before combining so it doesn't appear twice
         
        X_train_poly = X_train_poly.drop(columns = [col])
        X_val_poly = X_val_poly.drop(columns = [col])
        X_train_poly = pd.concat([X_train_poly,X_train_poly_df], axis = 1)
        X_val_poly = pd.concat([X_val_poly,X_val_poly_df], axis = 1)
    
        # Find r-squared score on validation
        model_poly = lr.fit(X_val_poly, y_val)
        r_squared = model_poly.score(X_val_poly, y_val)
        
        
        # Append to data structure
        poly_results.append((col,degree,r_squared))
        poly_results_df = pd.DataFrame(poly_results,columns =['col_name','degree','R_squared'])

# Sort and subset the data structure to find the top 7
top_polynomials = poly_results_df.nlargest(7,'R_squared')
top_polynomials

,col_name,degree,R_squared
20,GrLivArea,4,0.856139
19,GrLivArea,3,0.850194
11,TotalBsmtSF,4,0.846229
10,TotalBsmtSF,3,0.846054
14,1stFlrSF,4,0.839917
2,LotArea,4,0.838375
13,1stFlrSF,3,0.835703


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [40]:
# Your code here

# Filter out duplicates
top_polynomials = top_polynomials.loc[top_polynomials.groupby('col_name')['R_squared'].idxmax()]
# Loop over remaining results
# Initialize new DataFrames for the best polynomial features
X_train_final = X_train_scaled.copy()
X_val_final = X_val_scaled.copy()

# Loop over remaining results
for index, row in top_polynomials.iterrows():
    col = row['col_name']
    degree = row['degree']

    # Instantiate PolynomialFeatures and transform the original column
    poly = PolynomialFeatures(degree=degree)
    X_train_col = X_train_final[[col]]
    X_val_col = X_val_final[[col]]

    col_X_train_poly = poly.fit_transform(X_train_col)
    col_X_val_poly = poly.transform(X_val_col)

    # Create DataFrames for polynomial features
    poly_feature_names = poly.get_feature_names_out([col])
    X_train_poly_df = pd.DataFrame(col_X_train_poly, columns=poly_feature_names)
    X_val_poly_df = pd.DataFrame(col_X_val_poly, columns=poly_feature_names)

    # Drop original column and concatenate new polynomial features
    #Concat new polynomials to X_train and X_val
    X_train_final = X_train_final.drop(columns=[col])
    X_val_final = X_val_final.drop(columns=[col])
    X_train_final = pd.concat([X_train_final, X_train_poly_df], axis=1)
    X_val_final = pd.concat([X_val_final, X_val_poly_df], axis=1)
    
    

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [41]:
# Your code here
X_train_final.head()

,OverallQual,OverallCond,2ndFlrSF,TotRmsAbvGrd,GarageArea,Fireplaces,1,1stFlrSF,1stFlrSF^2,1stFlrSF^3,...,1,LotArea,LotArea^2,LotArea^3,LotArea^4,1,TotalBsmtSF,TotalBsmtSF^2,TotalBsmtSF^3,TotalBsmtSF^4
0,-0.099842,-0.509252,1.261552,0.250689,0.327629,-0.994820,1.0,-0.804789,0.647685,-0.521250,...,1.0,-0.114710,0.013158,-0.001509,1.731453e-04,1.0,-0.639316,0.408725,-0.261304,1.670559e-01
1,0.632038,-0.509252,-0.808132,-0.365525,0.079146,-0.994820,1.0,0.641608,0.411661,0.264125,...,1.0,-0.176719,0.031230,-0.005519,9.752976e-04,1.0,0.838208,0.702592,0.588918,4.936357e-01
2,-0.831723,1.304613,-0.808132,-0.981739,-1.105931,-0.994820,1.0,-0.329000,0.108241,-0.035611,...,1.0,-0.246336,0.060682,-0.014948,3.682263e-03,1.0,-0.012560,0.000158,-0.000002,2.488547e-08
3,-0.831723,1.304613,-0.808132,-0.981739,-1.134602,0.588023,1.0,-0.609036,0.370925,-0.225906,...,1.0,-0.378617,0.143351,-0.054275,2.054946e-02,1.0,-0.339045,0.114951,-0.038974,1.321384e-02
4,-1.563603,1.304613,0.550523,0.250689,-2.281450,-0.994820,1.0,-1.315922,1.731650,-2.278716,...,1.0,-0.010898,0.000119,-0.000001,1.410729e-08,1.0,-2.531499,6.408489,-16.223084,4.106873e+01


In [42]:
X_val_final.columns


Index(['OverallQual', 'OverallCond', '2ndFlrSF', 'TotRmsAbvGrd', 'GarageArea',
       'Fireplaces', '1', '1stFlrSF', '1stFlrSF^2', '1stFlrSF^3', '1stFlrSF^4',
       '1', 'GrLivArea', 'GrLivArea^2', 'GrLivArea^3', 'GrLivArea^4', '1',
       'LotArea', 'LotArea^2', 'LotArea^3', 'LotArea^4', '1', 'TotalBsmtSF',
       'TotalBsmtSF^2', 'TotalBsmtSF^3', 'TotalBsmtSF^4'],
      dtype='object')

## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [44]:
# Your code here
lr.fit(X_val_final,y_val)
val_rsquared = lr.score(X_val_final,y_val)
val_rsquared

0.8687696351333026

In [45]:
lr.fit(X_train_final,y_train)
train_rsquared = lr.score(X_train_final,y_train)
train_rsquared

0.8544252677229207

It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [48]:
# Your code here
from sklearn.feature_selection import RFE

model = LinearRegression()

features = [10,15,5,20]


for feature in features:
    rfe = RFE(estimator = model, n_features_to_select = feature)
    X_train_selected = rfe.fit_transform(X_train_final,y_train)
    X_val_selected = rfe.transform(X_val_final)
    model.fit(X_train_selected,y_train)
    train_score = model.score(X_train_selected,y_train)
    val_score = model.score(X_val_selected, y_val[:X_val_selected.shape[0]])
    

    print(f"Number of features selected: {feature}")
    print(f"Training Score (R²): {train_score:.4f}")
    print(f"Validation Score (R²): {val_score:.4f}")
    print(f"Remaining features after RFE: {X_train_selected.shape[1]}\n")
    

Number of features selected: 10
Training Score (R²): 0.7907
Validation Score (R²): 0.3903
Remaining features after RFE: 10

Number of features selected: 15
Training Score (R²): 0.8496
Validation Score (R²): -33.6129
Remaining features after RFE: 15

Number of features selected: 5
Training Score (R²): 0.7760
Validation Score (R²): 0.6353
Remaining features after RFE: 5

Number of features selected: 20
Training Score (R²): 0.8536
Validation Score (R²): -13.0914
Remaining features after RFE: 20



Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [53]:
# Your code here
from sklearn.linear_model import Lasso


alphas = [10,15,8,10]

for alpha in alphas:
    model_lasso = Lasso(alpha = alpha)
    model_lasso.fit(X_train_final,y_train)
    lasso_train_score = model_lasso.score(X_train_final,y_train)
    lasso_val_score = model_lasso.score(X_val_final,y_val)


    print(f"Lasso Training Score (R²): { lasso_train_score:.4f}")
    print(f"Lasso Validation Score (R²): {lasso_val_score:.4f}")


Lasso Training Score (R²): 0.8544
Lasso Validation Score (R²): -11.2790
Lasso Training Score (R²): 0.8544
Lasso Validation Score (R²): -11.3036
Lasso Training Score (R²): 0.8544
Lasso Validation Score (R²): -11.2691
Lasso Training Score (R²): 0.8544
Lasso Validation Score (R²): -11.2790


Compare the results. Which features would you choose to use?

In [ ]:
# Your written answer here

## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [ ]:
# Your code here


### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [ ]:
# Your code here


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.